# **StomataPy - training**

Contact: hongyuan.zhang@usys.ethz.ch

The file structure:
```
├── StomataPy (the root directory)
    ├── Stomata_detection (data for object detection)
    ├── Stomata_segmentation (data for semantic segmentation)
    ├── train (config files for training)
    ├── Training.ipynb (the file you are running now)
```
There will be one point you need to run `export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation`, change the `StomataPy` to your `root_dir`

# I\/O operations

In [ ]:
!unzip Stomata_segmentation.zip -d Stomata_segmentation

In [ ]:
!zip -r Stomata_segmentation.zip Stomata_segmentation

# Check GPUs and CPUs

In [1]:
!pip install -U openmim
!mim install mmengine  # mmengine==0.8.3
!mim install 'mmcv>=2.0.0rc3'
!mim install "mmpretrain[multimodal]>=1.0.0rc8"
!pip install wandb memory_profiler psutil fairscale

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.27.1 requires requests>=2.31, but you have requests 2.28.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html
Defaulting to user installation because normal site-packages is not writ

In [2]:
import os
import mmcv
import mmengine
import torch

if torch.cuda.is_available():
    print('CUDA is available')
    device_count = torch.cuda.device_count()
    print(f'Total number of CUDA devices: {device_count}')
    for i in range(device_count):
        device_name = torch.cuda.get_device_name(i)
        print(f'Device {i}: {device_name}')
else:
    print('CUDA is not available')

print('\nNumber of workers: ')
!nproc

CUDA is available
Total number of CUDA devices: 4
Device 0: NVIDIA GeForce RTX 4090
Device 1: NVIDIA GeForce RTX 4090
Device 2: NVIDIA GeForce RTX 4090
Device 3: NVIDIA GeForce RTX 4090

Number of workers: 
256


# Check the config file (optional)

In [17]:
from mmengine import Config
cfg = Config.fromfile('Configs//swin-l_dino.py')
print(cfg.pretty_text)
#cfg.dump('Configs//swin-l_dino_config.py')

# MMDetection

## install mmdetection

In [ ]:
####### code block 1 ########
!mim install "mmdet>=3.0.0rc0"
#!git clone https://github.com/open-mmlab/mmdetection.git
#%cd mmdetection
#!pip install -v -e .
#%cd ..

In [ ]:
!mim search mmdet --model ddq

In [ ]:
!mim download mmdet --config ddq-detr-5scale_r50_8xb2-12e_coco --dest Configs

## visualize dataset

In [ ]:
mim run mmdet browse_dataset train/mmdetection/Swin-l_ddq_config.py  --output-dir viz_dataset_mmdet/ 
mim run mmdet browse_dataset train/mmdetection/Swin-l_dino_config.py  --output-dir viz_dataset_mmdet/ 
mim run mmdet browse_dataset train/mmdetection/Swin-s_mask2former_config.py  --output-dir viz_dataset_mmdet/ 

## training #e98d0d0fc185629c362c4f0f39cc9282d955d217

In [ ]:
mim train mmdet train/mmdetection/Swin-l_ddq_config.py --gpus 4 --launcher pytorch
mim train mmdet train/mmdetection/Swin-l_dino_config.py --gpus 4 --launcher pytorch
mim train mmdet train/mmdetection/Swin-s_mask2former_config.py --gpus 4 --launcher pytorch

# Rein

## install mmsegmentation and clone rein

In [2]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -v -e .
%cd ..
!pip install wandb
!git clone https://github.com/w1oves/Rein.git

fatal: destination path 'mmsegmentation' already exists and is not an empty directory.
/StomataPy/mmsegmentation


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Using pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///StomataPy/mmsegmentation
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-5u9hwo6a/mmsegmentation.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-5u9hwo6a/mmsegmentation.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-5u9hwo6a/mmsegmentation.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation
    Running command python setup.py develop
    running develop
    /usr/local/lib/python3.10/dist-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
      warnings.warn(
    /usr/local/lib/python3.10/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup

In [6]:
%cd ..

/StomataPy


In [21]:
import shutil
from tqdm import tqdm

list2copy = [
    'train/mmsegmentation/__init__.py',
    'train/mmsegmentation/stomata_dataset.py',
    'train/mmsegmentation/cell_dataset.py'
]    

for dir in tqdm(list2copy, total=len(list2copy)):
    shutil.copy2(dir, 'mmsegmentation/mmseg/datasets')

shutil.copy2('train/mmsegmentation/local_visualizer.py', 'mmsegmentation/mmseg/visualization/local_visualizer.py')

shutil.copy2('train/rein/reins_sam_vit.py', 'Rein/rein/models/backbones')
shutil.copy2('train/rein/rein_dinov2_mask2former_config.py', 'Rein/rein_dinov2_mask2former_config.py')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2701.94it/s]


'Rein/rein_dinov2_mask2former_config.py'

## preview data augmentation (run in terminal)

In [ ]:
cd Rein
export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation
python ../mmsegmentation/tools/analysis_tools/browse_dataset.py \
    rein_dinov2_mask2former_config.py  \
    --output-dir ../viz_dataset/ 

## get dinov2 weights

In [15]:
%cd Rein
!wget -P checkpoints https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth
!python tools/convert_models/convert_dinov2.py checkpoints/dinov2_vitl14_pretrain.pth checkpoints/dinov2_converted.pth --height 512 --width 512
%cd ..

/StomataPy/Rein
--2024-05-20 09:36:36--  https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth
Resolving proxy.ethz.ch (proxy.ethz.ch)... 129.132.202.155
Connecting to proxy.ethz.ch (proxy.ethz.ch)|129.132.202.155|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1217586395 (1.1G) [binary/octet-stream]
Saving to: ‘checkpoints/dinov2_vitl14_pretrain.pth.5’

dinov2_vitl14_pretr 100%[===================>]   1.13G   112MB/s    in 10s     

2024-05-20 09:36:46 (112 MB/s) - ‘checkpoints/dinov2_vitl14_pretrain.pth.5’ saved [1217586395/1217586395]

Load from checkpoints/dinov2_vitl14_pretrain.pth
Convert conv kernel in patch embed layer: torch.Size([1024, 3, 14, 14]) -> torch.Size([1024, 3, 16, 16])
Convert pos embedding: torch.Size([1, 1369, 1024]) -> (-1, 37, 37, 1024) -> torch.Size([1, 1024, 32, 32]) -> torch.Size([1, 1024, 1024])
Save to checkpoints/dinov2_converted.pth
/StomataPy


## training (run in terminal)
wandb API: e98d0d0fc185629c362c4f0f39cc9282d955d217

In [2]:
# --cfg-options efficient_conv_bn_eval="[backbone]"

In [ ]:
export PYTHONPATH=$PYTHONPATH:/StomataPy/mmsegmentation
PORT=12345 CUDA_VISIBLE_DEVICES=0,1,2,3 bash tools/dist_train.sh rein_dinov2_mask2former_config.py 4

## generate full weights after training

In [ ]:
!python tools/generate_full_weights.py \
    --dinov2_segmentor_path checkpoints/dinov2_segmentor_test.pth \
    --backbone checkpoints/dinov2_vitl14_pretrain.pth \
    --rein_head checkpoints/best_mIoU_epoch_155.pth

## inference

In [ ]:
import torch
import torch.nn as nn
import rein
from mmengine.config import Config
from mmseg.apis.inference import inference_model,show_result_pyplot,init_model
import matplotlib.pyplot as plt
cfg=Config.fromfile('configs/dinov2/rein_dinov2_mask2former_test.py')
network:nn.Module=init_model(cfg,'checkpoints/dinov2_segmentor_test.pth','cuda:0')
network.cfg=cfg
#network.cfg.test_pipeline=network.cfg.bdd_test_pipeline
torch.set_grad_enabled(False)

In [ ]:
im_path='VegAnn_326.png'
result=inference_model(network,im_path)
vis_img=show_result_pyplot(network,im_path,result)
plt.imshow(vis_img)
plt.axis('off')